In [152]:
#import important stuff
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
from ase.spectrum.band_structure import BandStructurePlot
from ase.spectrum.band_structure import BandStructure
from ase.build import mx2
import numpy as np
import matplotlib.pyplot as plt

In [153]:
formula = 'MoS2'
sym_path='KG'
nP=30
emptybands=13
bs1=BandStructure.read('formula_'+formula+'_direction_'+sym_path+'_npoints_{}_emptybands_{}'.format(nP,emptybands)+'.json')

In [143]:
bs1

BandStructure(path=BandPath(path='Kpt0G', cell=[3x3], special_points={AGHKKpt0LM}, kpts=[60x3]), energies=[1x60x39 values], reference=-3.221897466544023)

In [72]:
#bs1_dens

BandStructure(path=BandPath(path='Kpt0G', cell=[3x3], special_points={AGHKKpt0LM}, kpts=[2x3]), energies=[1x2x26 values], reference=-3.221015198565162)

In [144]:
Nkpt=np.shape(bs1.energies)[1]
kPath=bs1.path.kpts
kDiff=kPath-kPath[0]
DeltaK_mag=np.zeros((Nkpt))
test_K=np.zeros((Nkpt))
for i in range(Nkpt):
    DeltaK_mag[i]=np.sqrt((kPath[i,0]-kPath[0,0])**2+(kPath[i,1]-kPath[0,1])**2+(kPath[i,2]-kPath[0,2])**2)
    test_K[i]=np.linalg.norm(kDiff[i])
#print(DeltaK_mag)
#print(test_K)#regner rigtig:)

In [145]:
energybands=bs1.energies[0]
np.shape(energybands)
valenceBand=energybands[:,12]
conductionBand=energybands[:,13]

In [146]:
#choose a cut off
k_cut=0.015
bool_k=DeltaK_mag<k_cut
DeltaK_cut=DeltaK_mag[bool_k]
valenceBand_cut=valenceBand[bool_k]
conductionBand_cut=conductionBand[bool_k]

In [147]:
kVec=np.concatenate((np.sort(-DeltaK_cut),DeltaK_cut[1:]))
valVec=np.concatenate((np.sort(valenceBand_cut),valenceBand_cut[1:]))
conVec=np.concatenate((-np.sort(-conductionBand_cut),conductionBand_cut[1:]))
print(kVec)
print(valVec)
print(conVec)

[-0.01263315 -0.          0.01263315]
[-4.0642329  -4.06083408 -4.0642329 ]
[-2.39500485 -2.39920275 -2.39500485]


In [97]:
#units
import ase.units
Hartree = ase.units.Hartree#Hartree energy
Bohr = ase.units.Bohr#Bohr radius
print(Hartree)
print(Bohr)

27.211386024367243
0.5291772105638411


In [148]:
#fit
from scipy.optimize import curve_fit
xData=kVec
y1Data=valVec
y2Data=conVec
#fit function
atomUnit_to_eV=Hartree
Aang_to_atomUnit=1/Bohr
#apparently k is already in atomic units!!!
def val_band(x,A1,m_eff_h):
    return A1-atomUnit_to_eV*(x)**2/(2*m_eff_h)
def con_band(x,A1,m_eff_e):
    return A1+atomUnit_to_eV*(x)**2/(2*m_eff_e)
parameters1, _ = curve_fit(val_band, xData, y1Data)
parameters2, _ = curve_fit(con_band, xData, y2Data)
print(parameters1)
print(parameters2)
fit_val=val_band(xData,parameters1[0],parameters1[1])
fit_con=con_band(xData,parameters2[0],parameters2[1])
print(fit_val)
#print(valVec)
print(fit_con)

[-4.06083408  0.63887512]
[-2.39920275  0.51726437]
[-4.0642329  -4.06083408 -4.0642329 ]
[-2.39500485 -2.39920275 -2.39500485]


In [134]:
plt.plot(kVec,valVec,'r-*')
plt.plot(kVec,fit_val,'b-*')
plt.legend(['Data','Fit'])
plt.xlabel('Delta k')
plt.ylabel('E [eV]')
plt.title('Effective h-mass: {}'.format(parameters1[1]))
plt.savefig('./plots/valFit_{}_sym_{}_npoints_{}_emptybands{}.svg'.format(formula,sym_path,nP,emptybands))
plt.close()

plt.plot(kVec,conVec,'r-*')
plt.plot(kVec,fit_con,'b-*')
plt.legend(['Data','Fit'])
plt.xlabel('Delta k')
plt.ylabel('E [eV]')
plt.title('Effective e-mass: {}'.format(parameters2[1]))
plt.savefig('./plots/conFit_{}_sym_{}_npoints_{}_emptybands_{}.svg'.format(formula,sym_path,nP,emptybands))
plt.close()

In [68]:
print(DeltaK_mag[1:]-DeltaK_mag[:len(DeltaK_mag)-1])

[0.02570193 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193
 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193
 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193
 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193
 0.02570193 0.02570193 0.02570193 0.02570193 0.02570193]


In [139]:
print((DeltaK_mag[1:]-DeltaK_mag[:len(DeltaK_mag)-1])[0]*2)#for KM med 30 (stort set halvt så stor som spacing i den anden retning med 60)

0.025266304830506115


In [149]:
#KG med 59
print((DeltaK_mag[1:]-DeltaK_mag[:len(DeltaK_mag)-1])[0]*2)

0.025266304830506115


In [ ]:
plt.plot(DeltaK_mag,valenceBand)
plt.plot(DeltaK_mag,conductionBand)
plt.legend(['Valence band','Conduction band'])
plt.xlabel('Delta k')
plt.ylabel('E')
plt.show()

In [ ]:
plt.plot(DeltaK_cut,valenceBand_cut)
plt.plot(DeltaK_cut,conductionBand_cut)
plt.legend(['Valence band','Conduction band'])
plt.xlabel('Delta k')
plt.ylabel('E')
plt.show()

In [ ]:
plt.plot(kVec,valVec)
plt.plot(kVec,conVec)
plt.legend(['Valence band','Conduction band'])
plt.xlabel('Delta k')
plt.ylabel('E')
plt.show()